# 一般性能建议 —— 总结

基本常识：
+ 动态编译（dynamic compilation）指的是“在运行时进行编译”；与之相对的是事前编译（ahead-of-time compilation，简称AOT），也叫静态编译（static compilation）。

+ JIT编译（just-in-time compilation）狭义来说是当某段代码即将第一次被执行时进行编译，因而叫“即时编译”。JIT编译是动态编译的一种特例。JIT编译一词后来被泛化，时常与动态编译等价；但要注意广义与狭义的JIT编译所指的区别。

1. 编写类型稳定的代码
- Julia程序执行分为编译期和执行期，编译期就能确定变量类型的代码为类型稳定的代码， 如果必须要程序真正跑起来才能确定变量类型，那说明类型不稳定，可以用`@code_warntype`检查
- Julia的编译器会自动进行类型推断， 一般来说不需要进行类型标注，除了含糊不清的情况。
- 避免使用全局变量，使用全局常量，或者将该变量作为函数参数传入函数体
- 避免使用可变结构体，使用不可变结构体
  - 结构体内的参数必须要进行类型声明，否则编译期无法确定类型，会导致类型不稳定的情况
  - 结构体内的参数要通过参数化类型来声明为抽象类型的子类型
2. 避免内存分配
- 必要时使用数组的视图，而不是copy
- 避免使用向量化代码，而是使用循环
- 按列读取数组
  - CPU利用三级缓存从内存中读取连续内容，而恰好Julia中的数据是按照列进行存储的，所以在进行for循环时，按照列进行进行循环，会减少CPU从内存读取的次数，加速运算，效果比较明显
3. 其它
- 使用`@inbounds`避免数组边界检查，建议在for-loop中和`eachindex()`, `axes()`搭配食用
    - `@inbounds`可以和`@simd @threads`等一起使用，但是要放在最前边
- Julia中的条件判断
    - `ifelse(a > b, fun1, fun2)`, fun1和fun2都会被执行，等价于`fun1 * (a > b) + (1 - (a > b)) * fun2`，按道理讲这样计算会变慢，之所以存在这个函数，是因为`@simd`宏的使用必须要避免if判断，所以`ifelse`常常和`@simd`搭配使用，其它场景不太合适🚫
    - `if a > b fun1 else fun2 end`和三元运算符`a > b ? fun1 : fun2`，只会执行一个分支

# 工具箱

+ `@btime`(来自于BenchmarkTools包)用来判断程序执行时间和内存分配，julia自带的`@time`第一次执行加入了编译时间，而`@btime`得到的是多次计算取平均值，可能会比较慢
+ `@code_llvm`获取llvm源码
+ `@code_native`获汇编代码
+ `@code_warntype`查看是否类型稳定

In [1]:
using BenchmarkTools
using LoopVectorization
using VectorizedReduction # 基于LoopVectorization的高效压缩计算库
using StatsFuns

LoadError: ArgumentError: Package LoopVectorization not found in current path.
- Run `import Pkg; Pkg.add("LoopVectorization")` to install the LoopVectorization package.

# 内存优化

## 必要时使用数组视图而不是切片

In [2]:
a = rand(100);

In [3]:
@btime sum(a[1:20]);

  77.915 ns (2 allocations: 240 bytes)


In [4]:
@btime @views sum(a[1:20]);

  49.899 ns (2 allocations: 64 bytes)


In [5]:
a[1:10] .= 1.0

10-element view(::Vector{Float64}, 1:10) with eltype Float64:
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000
 1.0000

In [6]:
@btime a[end]

  41.877 ns (1 allocation: 16 bytes)


0.4629

In [7]:
@btime @view a[end]

  41.532 ns (1 allocation: 48 bytes)


0-dimensional view(::Vector{Float64}, 100) with eltype Float64:
0.4629

+ `a[1:10]`会copy数据，而`a[1:10] .= 1.0`是直接修改`a`，也可以说是在视图上修改，是十分高效的，使用视图都是针对第一种不是赋值修改的操作
  - 题外话：Julia大部分以`！`结尾的函数直接修改输入，采用的方法，很多就是`a[1:10] .= 1.0`，但是要注意的是，要避免重复创建数组，能用上一步已经创建好的数据，就不要创建新的，这个带来的提升是巨大的；
+ **数组的多个位置的索引(例如`a[100, :]`, `a[1:100, :]`索引一行或者一列甚至是不连续索引)使用视图，提升非常明显（几乎是1倍的提升）**
+ **单个点索引使用视图没有效果，反而会时程序运行更慢，原因大概是创建视图的时间要高于`getindex()`执行一次的时间**
+ `@view`与`@views`的区别
  - `@view`只能处理单个变量的索引，例如`+(@view a[1], a[2])`会报错，因为它把`, @a[2]`也传入给了`@view`;
  - `@view`必须和要索引的变量紧挨着，例如`@view sum(a[1:2])`就会报错，必须是`sum(@view a[1:2])`;
  - `@views`恰好解决了以上两个问题，例如`@views +(a[1], a[2])`，`@views`置于行首，那么该行都会使用视图进行切片；

## 按照列的方式索引矩阵

Julia按照coloumn-major的方式存储多维数组，所以按照列索引进行循环，程序的性能可以提高数倍相比于按照行索引。

原因在于：按照列索引是顺序阅读数据，可以利用CPU的cache，提高缓存命中率🎯

[参考资料](https://eli.thegreenplace.net/2015/memory-layout-of-multi-dimensional-arrays#:~:text=The%20row%2Dmajor%20layout%20of,%2C%20then%20the%20second%2C%20etc.)

![](https://eli.thegreenplace.net/images/2015/column-major-2D.png)

## 循环快于广播

R和Matlab的向量化代码比较快的原因是
+ 向量化代码用C语言展开为for-loop，不受到R或Matlab本身的限制
+ 底层使用C时，数值类型明确，可以触发更加高效的优化手段（如simd）

In [8]:
N = 1000
a = randn(N)
b = randn(N)
c = rand(N)
d = randn(N) * 2;

In [9]:
function testdot(a, b, c, d)
    return sum(a .* b .+ c./ d .- 1) 
end

testdot (generic function with 1 method)

In [10]:
@btime testdot($a, $b, $c, $d)

  701.852 ns (1 allocation: 7.94 KiB)


12.1627

In [11]:
function testloop(a, b, c, d)
    s = zero(eltype(a))
    @turbo for i in eachindex(a)
        s += a[i] * b[i] + c[i] / d[i] - 1
    end
    return s;
end

@btime testloop($a, $b, $c, $d)

LoadError: LoadError: UndefVarError: `@turbo` not defined
in expression starting at In[11]:3

注意这里的for-loop版本的计算很明显要快于向量化计算，原因在于这么几个方面
+ 向量化代码需要更高的内存占用，储存的中间变量为矩阵
+ for-loop使用`@inbounds`，避免了数组边界检查（配合`eachindex`可以确保不会越界）
+ for-loop使用`@simd`，CPU级别的并行，意思是`单指令多数据`，在有AVX指令的Intel CPU的效果最好，原理在于，寄存器较大，一次运算可以同时处理载入的4批数据（4 * 64= 256 bit），注意事项：用在单层循环，循环的顺序对结果没有影响时才可以使用
+ **不使用`@simd` `@inbounds`的循环性能不如向量化版本**

在实际使用中， 向量化操作简洁易懂， 如果把向量化操作写成for-loop的形式，可以单独把它写成一个函数，这样能让程序更简洁

# 编写类型稳定的代码

## Val()

Val可以给Julia编译器提供额外的类型信息

In [12]:
using StaticArrays

In [13]:
SVector{4, Float64}(1:4)

4-element SVector{4, Float64} with indices SOneTo(4):
 1.0000
 2.0000
 3.0000
 4.0000

In [14]:
function static(v::Vector)
    return SVector{length(v), eltype(v)}(v)
end

static (generic function with 1 method)

In [15]:
@code_warntype static([1, 2, 3.0])

MethodInstance for static(::Vector{Float64})
  from static(v::Vector) @ Main In[14]:1
Arguments
  #self#::Core.Const(static)
  v::Vector{Float64}
Body::Any
1 ─ %1 = Main.SVector::Core.Const(SVector)
│   %2 = Main.length(v)::Int64
│   %3 = Main.eltype(v)::Core.Const(Float64)
│   %4 = Core.apply_type(%1, %2, %3)::Type{SVector{_A, Float64}} where _A
│   %5 = (%4)(v)::Any
└──      return %5



为啥显示类型不稳定？程序在编译时仍然不知道length(v)是多少，运行时才知道，但是`eltype(v)`在编译时是知道的，julia能推断出`eltype(v)`, 从这大概可以看出，Julia编译器可以对类型进行推断，但无法对表达式进行腿推断

Julia对于`Val`结构体的定义, 可以看出`Val(X)`实例化了一个结构体， 得到`Val{X}()`
> ```julia
struct Val{X} where X end
Val(X) = Val{X}()
```

可以看出`Val`既是一个参数化的结构体`Val{}`，也是一个构造函数`Val()`。与常规结构体不同的是，`Val{x}`中的`x`不一定是一个`类型`，也可以是值。因为这个结构体中没有任何的field

In [16]:
Val(1), Val(Int64)

(Val{1}(), Val{Int64}())

In [17]:
function static2(v::Vector, ::Val{l}) where {l}
    return SVector{l, eltype(v)}(v)
end

static2 (generic function with 1 method)

In [18]:
function static3(v::Vector, l::Int64)
    return SVector{l, eltype(v)}(v)
end

static3 (generic function with 1 method)

In [19]:
@code_warntype static2([1, 2, 3], Val(3))

MethodInstance for static2(::Vector{Int64}, ::Val{3})
  from static2(v::Vector, ::Val{l}) where l @ Main In[17]:1
Static Parameters
  l = 3
Arguments
  #self#::Core.Const(static2)
  v::Vector{Int64}
  _::Core.Const(Val{3}())
Body::SVector{3, Int64}
1 ─ %1 = Main.SVector::Core.Const(SVector)
│   %2 = $(Expr(:static_parameter, 1))::Core.Const(3)
│   %3 = Main.eltype(v)::Core.Const(Int64)
│   %4 = Core.apply_type(%1, %2, %3)::Core.Const(SVector{3, Int64})
│   %5 = (%4)(v)::SVector{3, Int64}
└──      return %5



上边的代码为啥类型稳定了？ “它将值l作为编译器就可以知道的类型信息告诉julia了” -JonnyChen

In [20]:
@code_warntype static3([1, 2, 3], 3)

MethodInstance for static3(::Vector{Int64}, ::Int64)
  from static3(v::Vector, l::Int64) @ Main In[18]:1
Arguments
  #self#::Core.Const(static3)
  v::Vector{Int64}
  l::Int64
Body::Any
1 ─ %1 = Main.SVector::Core.Const(SVector)
│   %2 = Main.eltype(v)::Core.Const(Int64)
│   %3 = Core.apply_type(%1, l, %2)::Type{SVector{_A, Int64}} where _A
│   %4 = (%3)(v)::Any
└──      return %4



这个为啥类型不稳定呢？ 貌似3是作为一个变量传进去的， 在llvm源码中看到 `_A = 3`这个代码， 应该是要先执行这个代码， 再去生成`SVector（）`

In [21]:
struct VecS
    x::Vector{Int64}
    y::Int64
end

function static4(d::VecS)
    return SVector(d.x, d.y)
end

@code_warntype static4(VecS([1, 2, 3], 3))

MethodInstance for static4(::VecS)
  from static4(d::VecS) @ Main In[21]:6
Arguments
  #self#::Core.Const(static4)
  d::VecS
Body::SVector{2, Any}
1 ─ %1 = Base.getproperty(d, :x)::Vector{Int64}
│   %2 = Base.getproperty(d, :y)::Int64
│   %3 = Main.SVector(%1, %2)::Core.PartialStruct(SVector{2, Any}, Any[Tuple{Vector{Int64}, Int64}])
└──      return %3



## 避免声明容器内元素为抽象类型

容器包含：数组，矩阵，向量，元组，命名元组，结构体，自定义结构体等

In [22]:
# apple必须是抽象类型
struct badstructs
    apple::AbstractString 
end

In [23]:
# apple是AbstractString的子类型即可，实例化时Julia会自动选择范围最小的那个具体类型
struct goodstructs{T<:AbstractString}
    apple::T 
end

In [24]:
typeof(badstructs("apple"))

badstructs

In [25]:
typeof(goodstructs("apple"))

goodstructs{String}

## 不要在函数内部使用全局变量

### 函数内部使用全局变量(Bad)

In [26]:
# Bad way
I = 100
x = randn(I)
function sumI1()
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI1 (generic function with 1 method)

In [27]:
@code_warntype sumI1()

MethodInstance for sumI1()
  from sumI1() @ Main In[26]:4
Arguments
  #self#::Core.Const(sumI1)
Locals
  @_2::Any
  s::Any
  i::Any
Body::Any
1 ─ %1  = Main.eltype(Main.x)::Any
│         (s = Main.zero(%1))
│   %3  = (1:Main.I)::Any
│         (@_2 = Base.iterate(%3))
│   %5  = (@_2 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_2::Any
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Any
│   %11 = s::Any
│   %12 = Base.getindex(Main.x, i)::Any
│         (s = %11 + %12)
│         (@_2 = Base.iterate(%3, %10))
│   %15 = (@_2 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s



`I`和`x`是全局变量， 函数内部使用了全局变量， 运行时程序不会报错， 但是出现了类型不稳定的情况，原因是`s`和`I`在编译时不知道类型， 只有在执行时才知道

正确的做法：
+ Method 1（不推荐）: 声明I和x为全局常量，`const I = 100`, 但是不推荐，因为Julia REPL中更改全局常量会警告你`有可能会修改失败`
+ Method 2（推荐）：将I和x以参数的形式传入到函数中

In [28]:
# Method 2 demo
function sumI2(I, x)
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI2 (generic function with 1 method)

In [29]:
@code_warntype sumI2(I, x)

MethodInstance for sumI2(::Int64, ::Vector{Float64})
  from sumI2(I, x) @ Main In[28]:2
Arguments
  #self#::Core.Const(sumI2)
  I::Int64
  x::Vector{Float64}
Locals
  @_4::Union{Nothing, Tuple{Int64, Int64}}
  s::Float64
  i::Int64
Body::Float64
1 ─ %1  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%1))
│   %3  = (1:I)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_4 = Base.iterate(%3))
│   %5  = (@_4 === nothing)::Bool
│   %6  = Base.not_int(%5)::Bool
└──       goto #4 if not %6
2 ┄ %8  = @_4::Tuple{Int64, Int64}
│         (i = Core.getfield(%8, 1))
│   %10 = Core.getfield(%8, 2)::Int64
│   %11 = s::Float64
│   %12 = Base.getindex(x, i)::Float64
│         (s = %11 + %12)
│         (@_4 = Base.iterate(%3, %10))
│   %15 = (@_4 === nothing)::Bool
│   %16 = Base.not_int(%15)::Bool
└──       goto #4 if not %16
3 ─       goto #2
4 ┄       return s



### 函数通过结构体传入数据

避免使用可变的`mutable struct`而是使用不可变的`struct`，不可变结构体放在stack上，内存寻址快，可变结构体放在heap上，访问慢， 不可变的是最高效的

In [36]:
# 结构体内不声明参数类型，编译期不知道fields是什么类型，执行时才知道
struct data
    I
    x
end
function sumI3(d::data)
    x, I = d.x, d.I
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI3 (generic function with 1 method)

In [43]:
d = data(10, rand(10))
@code_warntype sumI3(d)

MethodInstance for sumI3(::data)
  from sumI3(d::data) @ Main In[36]:6
Arguments
  #self#::Core.Const(sumI3)
  d::data
Locals
  @_3::Any
  s::Any
  I::Any
  x::Any
  i::Any
Body::Any
1 ─ %1  = Base.getproperty(d, :x)::Any
│   %2  = Base.getproperty(d, :I)::Any
│         (x = %1)
│         (I = %2)
│   %5  = Main.eltype(x)::Any
│         (s = Main.zero(%5))
│   %7  = (1:I)::Any
│         (@_3 = Base.iterate(%7))
│   %9  = (@_3 === nothing)::Bool
│   %10 = Base.not_int(%9)::Bool
└──       goto #4 if not %10
2 ┄ %12 = @_3::Any
│         (i = Core.getfield(%12, 1))
│   %14 = Core.getfield(%12, 2)::Any
│   %15 = s::Any
│   %16 = Base.getindex(x, i)::Any
│         (s = %15 + %16)
│         (@_3 = Base.iterate(%7, %14))
│   %19 = (@_3 === nothing)::Bool
│   %20 = Base.not_int(%19)::Bool
└──       goto #4 if not %20
3 ─       goto #2
4 ┄       return s



In [44]:
# 参数化类型
# Notes. 不要将容易内的元素标注为抽象类型，而是要使用参数化类型将其标注为抽象类型的子类型
struct data2{T <: Signed, Y <: AbstractVector{<:AbstractFloat}}
    I::T            
    x::Y
end
function sumI4(d::data2)
    x, I = d.x, d.I
    s = zero(eltype(x))
    for i in 1:I
        s+=x[i]
    end
    return s
end

sumI4 (generic function with 1 method)

In [45]:
@which data2(10, rand(10))

data2(I::T, x::Y) where {T<:Signed, Y<:(AbstractVector{<:AbstractFloat})}
     @ Main In[44]:4

In [46]:
# 实例化以后生成的I::Int64, x::Vector{Float64}
dump(data2(10, rand(10)))

data2{Int64, Vector{Float64}}
  I: Int64 10
  x: Array{Float64}((10,)) [0.8632, 0.9924, 0.8199, 0.4621, 0.1936, 0.7825, 0.7157, 0.8747, 0.7398, 0.6717]


In [47]:
@code_warntype sumI4(data2(10, rand(10)))

MethodInstance for sumI4(::data2{Int64, Vector{Float64}})
  from sumI4(d::data2) @ Main In[44]:7
Arguments
  #self#::Core.Const(sumI4)
  d::data2{Int64, Vector{Float64}}
Locals
  @_3::Union{Nothing, Tuple{Int64, Int64}}
  s::Float64
  I::Int64
  x::Vector{Float64}
  i::Int64
Body::Float64
1 ─ %1  = Base.getproperty(d, :x)::Vector{Float64}
│   %2  = Base.getproperty(d, :I)::Int64
│         (x = %1)
│         (I = %2)
│   %5  = Main.eltype(x)::Core.Const(Float64)
│         (s = Main.zero(%5))
│   %7  = (1:I)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(1), Int64])
│         (@_3 = Base.iterate(%7))
│   %9  = (@_3 === nothing)::Bool
│   %10 = Base.not_int(%9)::Bool
└──       goto #4 if not %10
2 ┄ %12 = @_3::Tuple{Int64, Int64}
│         (i = Core.getfield(%12, 1))
│   %14 = Core.getfield(%12, 2)::Int64
│   %15 = s::Float64
│   %16 = Base.getindex(x, i)::Float64
│         (s = %15 + %16)
│         (@_3 = Base.iterate(%7, %14))
│   %19 = (@_3 === nothing)::Bool
│   %20 = Base.not_i